<a href="https://colab.research.google.com/github/sunnyna/TIL/blob/master/bond_analysis_report_crawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

drive 연결

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 채권 분석 리포트 크롤링

In [31]:
import requests
from bs4 import BeautifulSoup

pdf_list = []
date_list = []
title_list = []

for i in range(23,127):
  url = 'https://finance.naver.com/research/debenture_list.nhn?'
  params = {
      'page' : i
  }

  resp = requests.get(url,params=params)
  soup = BeautifulSoup(resp.content, 'html.parser')

  td_tag = soup.findAll("td",{"class":"file"})
  date_tag = soup.findAll('td', {"style":"padding-left:5px"} )
  title_tag = soup.findAll('td', {"style":"padding-left:10px"})



  for t in td_tag:
    pdf_url=t.find('a').attrs['href']
    pdf_list.append(pdf_url)



  for date in date_tag:
    date_list.append(date.get_text())

  for title in title_tag:
    title_list.append(title.get_text())


갯수 확인 및 슬라이싱(08.04.01~17.12.27까지만)

In [32]:
print(len(pdf_list))   
print(len(date_list))
print(len(title_list))


print(date_list[-10:])
print(date_list[:10])

3099
3099
['08.05.20', '08.05.19', '08.05.13', '08.05.04', '08.04.28', '08.04.21', '08.04.14', '08.04.07', '08.04.07', '08.04.01']
['18.01.02', '18.01.02', '17.12.27', '17.12.26', '17.12.20', '17.12.20', '17.12.18', '17.12.15', '17.12.15', '17.12.14']


슬라이싱 된 것 확인

In [34]:

pdf_list_ = pdf_list[2:]
date_list_ = date_list[2:]
title_list_ = title_list[2:]

print(date_list_[-10:])
print(date_list_[:10])

['08.05.20', '08.05.19', '08.05.13', '08.05.04', '08.04.28', '08.04.21', '08.04.14', '08.04.07', '08.04.07', '08.04.01']
['17.12.27', '17.12.26', '17.12.20', '17.12.20', '17.12.18', '17.12.15', '17.12.15', '17.12.14', '17.12.14', '17.12.13']


In [35]:
print(len(pdf_list_))
print(len(date_list_))
print(len(title_list_))


3097
3097


### pdf 파일로 저장
path 주의해서 확인하기

In [ ]:

for pdf,title in zip(pdf_list_, title_list_):
  with open('/content/drive/My Drive/bond_analysis_report/{}.pdf'.format(title), "wb") as file:
    response = requests.get(pdf)
    file.write(response.content)

파일갯수가 리스트 갯수와 일치하는 지 확인 (겹치는 제목은 덮어쓰기 때문에 제대로 나왔는지 확인해야했음)

In [42]:
import os

path = "/content/drive/My Drive/bond_analysis_report"
file_list = os.listdir(path)

print ("file_list: {}".format(file_list))

len(file_list)

file_list: ['bond_analysis_report_17.12.27.pdf', 'bond_analysis_report_17.12.26.pdf', 'bond_analysis_report_17.12.20.pdf', 'bond_analysis_report_17.12.18.pdf', 'bond_analysis_report_17.12.15.pdf', 'bond_analysis_report_17.12.14.pdf', 'bond_analysis_report_17.12.13.pdf', 'bond_analysis_report_17.12.08.pdf', 'bond_analysis_report_17.12.07.pdf', 'bond_analysis_report_17.12.06.pdf', 'bond_analysis_report_17.12.04.pdf', 'bond_analysis_report_17.12.01.pdf', 'bond_analysis_report_17.11.30.pdf', 'bond_analysis_report_17.11.29.pdf', 'bond_analysis_report_17.11.28.pdf', 'bond_analysis_report_17.11.27.pdf', 'bond_analysis_report_17.11.23.pdf', 'bond_analysis_report_17.11.22.pdf', 'bond_analysis_report_17.11.21.pdf', 'bond_analysis_report_17.11.20.pdf', 'bond_analysis_report_17.11.17.pdf', 'bond_analysis_report_17.11.16.pdf', 'bond_analysis_report_17.11.15.pdf', 'bond_analysis_report_17.11.14.pdf', 'bond_analysis_report_17.11.13.pdf', 'bond_analysis_report_17.11.10.pdf', 'bond_analysis_report_17.1

1658

# pdf를 csv로 변환

In [36]:
!pip install tika
from tika import parser


  Created wheel for tika: filename=tika-1.24-cp36-none-any.whl size=32885 sha256=f0bb89bae7421db0840c32a2c1d51074b71da0fe24a68377e212291c5f398b90
  Stored in directory: /root/.cache/pip/wheels/73/9c/f5/0b1b738442fc2a2862bef95b908b374f8e80215550fb2a8975
Successfully built tika


1. pdf파일을 txt로 변환하는 함수정의

In [ ]:
def convert_pdf_to_txt(file_name):
  rawText = parser.from_file('{}'.format(file_name))
  rawData = rawText['content']

  return rawData.replace('\n','')

파일이름과 내용으로 이루어진 csv 파일 생성

In [ ]:
import csv
from pathlib import Path

folder = Path('/content/drive/My Drive/practice')
csv_file = Path('/content/drive/My Drive/bond_analysis_report_list.csv')

with csv_file.open('w', encoding='utf-8') as f:
    writer = csv.writer(f, csv.QUOTE_ALL)
    writer.writerow(['FileName', 'Text'])
    for pdf_file in folder.glob('*.pdf'):
        pdf_text = convert_pdf_to_txt(pdf_file)
        writer.writerow([pdf_file.name, pdf_text])
